In [144]:
import pandas as pd
from nltk import download
download('vader_lexicon')

In [145]:
from pymongo import MongoClient
client = MongoClient()
db = client['reddit-data']
col_sub = db['submissions']
submissions = pd.DataFrame(col_sub.find())
col_com = db['comments']
comments = pd.DataFrame(col_com.find())

In [183]:
def get_sentiment_data(ticker, itemtype):
    
    
    from pymongo import MongoClient
    client = MongoClient()
    
    db = client['reddit-data'] 
    
    
    if itemtype == 'comments':
        collection = db['comments']
        
    elif itemtype == 'submissions':
        collection = db['submissions']
        
    #df = pd.DataFrame(collection.find())    
    df = pd.read_csv(f'{itemtype}.csv',low_memory=False)
        
    columns = {'comments': 'body', 'submissions': 'selftext'}
    
    column = columns[itemtype]    
    
    df[column] = df[column].apply(lambda x: str(x).lower())
    
    df = pd.concat([df[df[column].str.contains(ticker.lower())], df[df[column].str.contains(TICKERS[ticker].lower())]],ignore_index=True)
    
    from nltk.corpus import stopwords
    additional  = ['rt','rts','retweet'] #we'll store additional stopwords here
    swords = set().union(stopwords.words('english'),additional) #big list containing all the stopwords + our additional ones
    df.drop_duplicates(subset=columns[itemtype],inplace=True)
    
    import nltk.sentiment.vader as vd
    
    sia = vd.SentimentIntensityAnalyzer()
    df.dropna(subset=[column],inplace=True)
    
    df['processed_text'] = df[column].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords])
    
    from nltk.tokenize import word_tokenize
    
    df['sentiment_score'] = df['processed_text'].apply(lambda x: sum([ sia.polarity_scores(i)['compound'] for i in word_tokenize( ' '.join(x) )]) )
    
    df[['processed_text','sentiment_score']].sort_values(by='sentiment_score')
    
    sent_clasification = pd.cut(df['sentiment_score'],\
          [-3,-1.2, 0, 1.2 , 3],\
          right=True,\
          include_lowest=True,\
          labels=['strongly negative', 'negative', 'positive', 'strongly positive'])
    
    return dict(sent_clasification.value_counts())    

In [125]:
from assets import TICKERS

In [190]:
def get_sentiment_aggregates(ticker):
    com_results = get_sentiment_data('MMM', 'comments' )
    sub_results = get_sentiment_data('MMM', 'submissions' )
    
    labels=['strongly negative', 'negative', 'positive', 'strongly positive']
    aggregation = {}
    
    for key in labels:
        try:
            aggregation[key] = com_results[key] + sub_results[key]
        except e:
            print(e)

    return aggregation